# 设置 URL 参数（params）

在发送 HTTP 请求时如何正确设置 URL 查询参数（query string）

## 目录
1. 使用 requests 的 `params` 参数（简单示例）
2. 合并已包含查询字符串的 URL 与 `params`
3. 列表、多值参数与 None 的行为
4. urllib.parse: urlencode 与手工构造查询字符串
5. 编码、保留字符与安全性


## 1. 使用 requests 的 `params` 参数（简单示例）
requests 提供了一个方便的 `params` 参数，用来传入 dict 或 list，库会自动把它编码到 URL 中。


In [ ]:
import json5
import requests

# 简单的 params 示例
params = {'q': 'py请求', 'page': 2}
r = requests.get('https://httpbin.org/get', params=params, timeout=10)
print('请求的 URL:', r.url)
print('请求返回的 args 字段:', json5.dumps(r.json().get('args'),ensure_ascii=False))


## 2. 合并已包含查询字符串的 URL 与 `params`
如果原始 URL 已带有查询字符串，requests 会合并它们（原有参数 + 新传入的 `params`）。

In [ ]:
# URL 已带查询字符串，params 会与之合并
r = requests.get('https://httpbin.org/get?x=1', params={'y': 2})
print('合并后 URL:', r.url)
print('请求返回的 args 字段:', json5.dumps(r.json().get('args'),ensure_ascii=False))


## 3. 列表、多值参数与 None 的行为
- 列表 / 多值：可以把参数的值设为 list，requests 会把它编码为重复的键（例如 key=a&key=b）。
- None 值：requests 会忽略值为 None 的参数（不会出现在最终 URL 中）。
- 布尔值/数字：会按字符串转换（例如 True -> 'True'）。


In [ ]:
# 列表参数示例
params = {'tag': ['python', 'requests'], 'empty': None, 'flag': True}
r = requests.get('https://httpbin.org/get', params=params)
print('URL:', r.url)
print('请求返回的 args 字段:', json5.dumps(r.json().get('args'),ensure_ascii=False))

# 注意：httpbin 在 args 中将重复键以最后一个值表现为字符串或以列表表现，取决于 httpbin 的版本和返回方式，实际行为以 response.json() 为准。


## 4. urllib.parse: urlencode 与手工构造查询字符串
有时候你可能需要手工构造查询字符串（例如需要特定的编码或对某些字符不进行编码），这时可以使用 `urllib.parse.urlencode` 或 `quote`。


In [ ]:
from urllib.parse import urlencode, quote_plus, quote

params = {'q': '中文 test', 'space': 'a b'}
# urlencode 会对非 ASCII 字符和空格进行转义（默认空格用 + 表示）
qs = urlencode(params)
print('urlencode ->', qs)
print('完整 URL ->', 'https://example.com/?' + qs)

# 如果你希望空格用 %20 而不是 +，可以用 quote 或 urlencode(..., quote_via=quote)（Python 3.5+ 可用）
qs_pct20 = urlencode(params, quote_via=quote)
print('使用 percent-encoding ->', qs_pct20)

# 手动对单个值编码（例如不想对某些字符编码）
safe_val = '你好/世界'
print('quote (保留 /):', quote(safe_val, safe='/'))


## 5. 编码、保留字符与安全性
- 浏览器与 HTTP 客户端对某些字符（比如空格、中文、+、/）的编码策略不同。requests 默认会做正确的 URL 编码，但若你需要精确控制请用 urllib.parse。
- 不要把已经编码好的查询字符串再传给 requests 的 `params`（会被二次编码）。如果你需要传入预编码字符串，可以直接把它拼接到 URL 上。
- 对于敏感信息（例如 token）尽量放在 header 中，而不是 URL 查询参数（URL 可能被记录在日志中）。




